## Git clone

Cloning this repository after you download this ipynb file.

In [9]:
import getpass, os

#token = getpass.getpass("input Token: ")
username = "qiulinfan"
owner = "qiulinfan"
repo = "graphNL2SQL"

#os.environ["GITHUB_TOKEN"] = token
#os.environ["GITHUB_USER"] = username
os.environ["GITHUB_OWNER"] = owner
os.environ["GITHUB_REPO"] = repo

!git clone https://github.com/${GITHUB_OWNER}/${GITHUB_REPO}.git
%cd graphNL2SQL/

Cloning into 'graphNL2SQL'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 125 (delta 44), reused 104 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 3.24 MiB | 18.83 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/graphNL2SQL/graphNL2SQL/graphNL2SQL


In [22]:
!git pull

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 8 (delta 7), reused 8 (delta 7), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 1.26 KiB | 322.00 KiB/s, done.
From https://github.com/qiulinfan/graphNL2SQL
   da4da48..3f76349  main       -> origin/main
Updating da4da48..3f76349
Fast-forward
 scripts/inference.py             | 10 +++++++++-
 scripts/prepare_training_data.py | 35 ++++++++++++++++++++++++-----------
 scripts/text_linearization.py    |  5 ++++-
 scripts/train.py                 | 11 ++++++++++-
 scripts/training_utils.py        |  7 ++++---
 5 files changed, 51 insertions(+), 17 deletions(-)


## Download and preprocess data

In [11]:
!python scripts/download_data.py

NL2SQL Dataset Downloader

[1/2] WikiSQL Dataset
----------------------------------------
WikiSQL: 100% 26.2M/26.2M [00:00<00:00, 198MB/s]
Extracting WikiSQL...
/content/graphNL2SQL/graphNL2SQL/graphNL2SQL/scripts/download_data.py:54: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(DATA_DIR)
WikiSQL extracted to /content/graphNL2SQL/graphNL2SQL/graphNL2SQL/data/wikisql

[2/2] Spider Dataset
----------------------------------------
Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=2b79add3-bbe1-4841-ad5c-8b3c1eb73473
To: /content/graphNL2SQL/graphNL2SQL/graphNL2SQL/data/spider_data.zip
100% 206M/206M [00:02<00:00, 91.5MB/s]
Extracting Spider dataset...
Renamed spider_data/ -> spider/
Spider extracted to 

In [12]:
!python scripts/prepare_training_data.py --semantic --semantic-threshold 0.8 --wikisql-balanced 2000 --spider

Processing WikiSQL train split...
Sampling 2000 balanced WikiSQL examples from train...
  Pattern distribution in dataset:
    where_only: 40598
    count: 5114
    min: 3231
    max: 3161
    avg: 2201
    sum: 2042
    select_only: 8
  Sampled 1722 examples
WikiSQL: 100% 1722/1722 [00:00<00:00, 41602.15it/s]
Processed 1722 WikiSQL examples
Processing WikiSQL dev split...
Sampling 400 balanced WikiSQL examples from dev...
  Pattern distribution in dataset:
    where_only: 6017
    count: 779
    max: 507
    min: 468
    avg: 329
    sum: 321
    select_only: 0
  Sampled 399 examples
WikiSQL: 100% 399/399 [00:00<00:00, 42613.75it/s]
Processed 399 WikiSQL examples
Saved 1722 examples to /content/graphNL2SQL/graphNL2SQL/graphNL2SQL/training_data/wikisql_train.jsonl
Saved 399 examples to /content/graphNL2SQL/graphNL2SQL/graphNL2SQL/training_data/wikisql_dev.jsonl
Processing Spider train split...
Spider:   0% 0/7000 [00:00<?, ?it/s]2025-12-12 15:08:22.344288: I tensorflow/core/util/port.c

In [13]:
import pandas as pd
pd.set_option("display.max_rows", None)   # 不省略行
pd.set_option("display.max_columns", None)  # 不省略列
pd.set_option("display.max_colwidth", None) # 不省略内容长度
df = pd.read_json("training_data/spider_train.jsonl", lines=True)
df['schema'][0:10]

,schema
0,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
1,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
2,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
3,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
4,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
5,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
6,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
7,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
8,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n head_ID (PK)\n name\n born_state\n age\nmanagement:\n department_ID (PK, FK)\n head_ID (FK)\n temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID\n\n[SEMANTIC LINKS]\ndepartment.Name ≈ head.name"
9,"[DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n Department_ID (PK)\n Name\n Creation\n Ranking\n Budget_in_Billions\n Num_Employees\nhead:\n h

In [14]:
import pandas as pd
pd.set_option("display.max_rows", None)   # 不省略行
pd.set_option("display.max_columns", None)  # 不省略列
pd.set_option("display.max_colwidth", None) # 不省略内容长度
df = pd.read_json("training_data/wikisql_train.jsonl", lines=True)
df['schema'][0:10]

,schema
0,[TABLES]\nCoolBrands_branding_initiative:\n Rank (PK)\n 2008\n 2009\n 2010\n 2011\n 2012\n 2013
1,[TABLES]\nUnited_States_House_of_Representatives_elections_1998:\n District (PK)\n Incumbent\n Party\n First elected\n Results\n Candidates
2,[TABLES]\nCircuit_Trois_Rivières:\n Year (PK)\n Date\n Driver\n Team\n Distance/Duration
3,[TABLES]\n1987_World_Rhythmic_Gymnastics_Championships:\n Place (PK)\n Name\n All Around\n Hoop\n Total
4,[TABLES]\nCentral_Murray_Football_League:\n Central Murray (PK)\n Wins\n Byes\n Losses\n Draws\n Against
5,[TABLES]\nCashmere_Mafia:\n Episode (PK)\n Rating\n Share\n Rating/Share (18-49)\n Viewers (millions)\n Rank (timeslot)\n Rank (night)\n Rank (week)
6,[TABLES]\n2008_Brazilian_Grand_Prix:\n Driver (PK)\n Constructor\n Laps\n Time/Retired\n Grid
7,[TABLES]\nUkraine_in_the_Eurovision_Song_Contest:\n Year(s) (PK)\n Television commentator\n Dual Television commentator\n Radio commentator\n Spokesperson
8,[TABLES]\n193940_New_York_Rangers_season:\n Game (PK)\n February\n Opponent\n Score\n Record
9,[TABLES]\n1987_World_Rhythmic_Gymnastics_Championships:\n Place (PK)\n Name\n All Around\n Hoop\n Total


## Training

### Install Dependencies

In [15]:
!pip install transformers datasets peft accelerate bitsandbytes torch wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.4 MB/s eta 0:00:00


In [16]:
!pip install -r requirements_train.txt

In [17]:
import transformers
import peft
import accelerate

print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"accelerate: {accelerate.__version__}")

transformers: 4.57.3
peft: 0.18.0
accelerate: 1.12.0


In [18]:
import os
import json
from pathlib import Path
from datetime import datetime

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)

import wandb

### GPU Check

In [19]:
import torch

print("=" * 60)
print("GPU INFORMATION")
print("=" * 60)

if torch.cuda.is_available():
    print(f"✓ CUDA available: True")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    total_mem = props.total_memory / 1e9
    print(f"  Memory: {total_mem:.2f} GB")
    print(f"  CUDA Version: {torch.version.cuda}")

    # Recommend config based on VRAM
    if total_mem < 12:
        print(f"\n  Recommended config: small_gpu (4-bit quantization)")
    elif total_mem < 24:
        print(f"\n Recommended config: default")
    else:
        print(f"\n Recommended config: large_gpu (more LoRA capacity)")
else:
    print("✗ CUDA not available - Training will be VERY slow on CPU")

print("=" * 60)

GPU INFORMATION
✓ CUDA available: True
  GPU: NVIDIA A100-SXM4-40GB
  Memory: 42.47 GB
  CUDA Version: 12.6

 Recommended config: large_gpu (more LoRA capacity)


### Configuration

Define hyperparameters and LoRA configuration. (No need to run this cell if config.json already good.)

In [35]:
import json
import sys
sys.path.insert(0, '.')  # Add project root to path

# =============================================================================
# CONFIGURATION - Edit values below, then run this cell to save
# =============================================================================

CONFIG = {
    "model": {
        "model_name": "Qwen/Qwen2.5-7B-Instruct",  # or "microsoft/Phi-3.5-mini-instruct"
        "load_in_4bit": True,   # Recommended for <16GB VRAM
        "load_in_8bit": False,
    },
    "lora": {
        "r": 16,                # Rank: 8, 16, 32, 64
        "alpha": 32,            # Typically 1-2x r
        "dropout": 0.1,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"],
    },
    "training": {
        "wikisql_epochs": 0,    # Phase 1: WikiSQL warmup
        "spider_epochs": 1.5,     # Phase 2: Spider main training
        "batch_size": 2,
        "gradient_accumulation": 8,  # Effective batch = batch_size * gradient_accumulation
        "learning_rate": 1e-4,
        "lr_scheduler": "cosine",
        "warmup_ratio": 0.05,
        "max_seq_length": 1024,
        "gradient_checkpointing": True,
        "use_bf16": True,
        "use_fp16": False,
    },
    "data": {
        "data_dir": "./training_data",
        "output_dir": "./checkpoints",
        "max_train_samples": None,  # None = use all
        "max_eval_samples": 500,
    },
    "wandb": {
        "enabled": True,
        "project": "nl2sql-finetuning",
        "run_name": None,  # Auto-generated if None
    },
    "save": {
        "strategy": "epoch",
        "total_limit": 5,
    },
}

# Save configuration to JSON file
with open('config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("Configuration saved to config.json")
print(f"  Model: {CONFIG['model']['model_name']}")
print(f"  LoRA rank: {CONFIG['lora']['r']}, alpha: {CONFIG['lora']['alpha']}")
print(f"  Batch size: {CONFIG['training']['batch_size']} x {CONFIG['training']['gradient_accumulation']} = {CONFIG['training']['batch_size'] * CONFIG['training']['gradient_accumulation']} effective")
print(f"  Training: {CONFIG['training']['wikisql_epochs']} epoch WikiSQL + {CONFIG['training']['spider_epochs']} epochs Spider")

Configuration saved to config.json
  Model: Qwen/Qwen2.5-7B-Instruct
  LoRA rank: 16, alpha: 32
  Batch size: 2 x 8 = 16 effective
  Training: 0 epoch WikiSQL + 1.5 epochs Spider


### Load Datasets & Run Training

Using encapsulated functions from `training_utils.py`

In [36]:
from scripts.training_utils import (
    TrainingConfig,
    load_config_from_json,
    save_config_to_json,
    print_config,
    load_datasets,
    show_sample_examples,
    load_model_and_tokenizer,
    setup_lora,
    print_model_parameters,
    init_wandb,
    train_phase1_wikisql,
    train_phase2_spider,
    finish_training,
)
print("Training utilities imported!")

Training utilities imported!


In [37]:
# Load configuration from JSON file
config = load_config_from_json('config.json')
print_config(config)

CONFIGURATION

Model: Qwen/Qwen2.5-7B-Instruct
  4-bit: True, 8-bit: False

LoRA:
  Rank: 16, Alpha: 32, Dropout: 0.1
  Target modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj']

Training:
  Batch size: 2 x 8 = 16 effective
  Learning rate: 0.0001, Scheduler: cosine
  Epochs: 0 WikiSQL + 1.5 Spider

Paths:
  Data: ./training_data
  Output: ./checkpoints

WandB: Enabled
  Project: nl2sql-finetuning


In [38]:
# Load datasets
wikisql_train, wikisql_dev, spider_train, spider_dev = load_datasets(config.data_dir)
show_sample_examples(wikisql_train, spider_train)

DATASET INFORMATION

 WikiSQL Dataset:
   Train samples: 1,722
   Dev samples:   399
   Columns: ['input', 'schema', 'question', 'output', 'sql', 'dataset', 'table_id', 'split']

 Spider Dataset:
   Train samples: 7,000
   Dev samples:   1,034
   Columns: ['input', 'schema', 'question', 'output', 'sql', 'dataset', 'db_id', 'split', 'num_tables', 'has_join']
   Multi-table: 7,000 (100.0%)
   With JOIN:   2,783 (39.8%)

SAMPLE EXAMPLES

 WikiSQL Example:
Question: What is the highest rank of the 2009 xbox?
SQL: SELECT MAX("Rank") FROM "table" WHERE "2009" = 'xbox'

Schema preview:
[TABLES]
CoolBrands_branding_initiative:
    Rank (PK)
    2008
    2009
    2010
    2011
    2012
    2013

------------------------------------------------------------

 Spider Example:
Question: How many heads of the departments are older than 56 ?
SQL: SELECT count(*) FROM head WHERE age  >  56
Database: department_management

Schema preview:
[DATABASE]
department_management

[TABLES]
department:
    Depar

In [39]:
# Load model and setup LoRA
model, tokenizer = load_model_and_tokenizer(config)
model = setup_lora(model, config)
print_model_parameters(model)

LOADING MODEL

Model: Qwen/Qwen2.5-7B-Instruct
Using 4-bit quantization (QLoRA)

Loading tokenizer...
Loading model (this may take a few minutes)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


 Model loaded!
CONFIGURING LoRA

LoRA Configuration:
   Rank (r):        16
   Alpha:           32
   Dropout:         0.1
   Target modules:  ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj']

 LoRA adapters added!
MODEL PARAMETERS (AFTER LoRA)
trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643

 Detailed Parameter Count:
   Total parameters:       4,373,157,376 (4.37B)
   Trainable parameters:      20,185,088 (20.19M)
   Frozen parameters:      4,352,972,288 (4.35B)
   Trainable %:                  0.4616%

 LoRA Adapter Size:
   LoRA parameters:           20,185,088 (20.19M)


In [40]:
# Initialize WandB
run_name = init_wandb(config)

 WandB initialized! Run: nl2sql_20251212_160349


### Phase 1: WikiSQL Warmup

In [41]:
# Phase 1: WikiSQL Warmup Training
train_phase1_wikisql(model, tokenizer, wikisql_train, wikisql_dev, config, run_name)

Skipping Phase 1 (WikiSQL not available or epochs=0)


### Phase 2: Spider Main Training

In [42]:
# Phase 2: Spider Main Training
trainer = train_phase2_spider(model, tokenizer, spider_train, spider_dev, config, run_name)

PHASE 2: Spider Main Training

Preparing Spider data...


Tokenizing Spider train:   0%|          | 0/7000 [00:00<?, ? examples/s]

Tokenizing Spider eval:   0%|          | 0/500 [00:00<?, ? examples/s]

/content/graphNL2SQL/graphNL2SQL/graphNL2SQL/scripts/training_utils.py:812: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.



 Spider Dataset Ready:
   Train: 7,000 samples
   Eval:  500 samples

 Starting Phase 2 training...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.088500,1.032762



 Saving Phase 2 model to ./checkpoints/phase2_spider/final

 Phase 2 complete!


### Finish Training

In [44]:
# Finish training and cleanup
#finish_training(config)

In [46]:
# Save the final model
# Use output directory from config
OUTPUT_DIR = config.output_dir + "/phase2_spider/final"

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

# Finish WandB run
import wandb
if wandb.run:
    wandb.finish()
    print("WandB run finished. Check your dashboard for loss graphs!")

Model saved to ./checkpoints/phase2_spider/final


eval/loss,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
train/grad_norm,█▄▂▂▂▂▃▃▃▇▃▃▂▂▃▁▁▃▁▁▂▂▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁
train/learning_rate,▅████████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,1.03276
eval/runtime,66.8919


WandB run finished. Check your dashboard for loss graphs!


### Download to local

In [48]:
import shutil
DOWNLOAD_PATH = "./checkpoints/phase2_spider/final"

def zip_and_download(model_path):
    model_path = Path(model_path)
    if not model_path.exists():
        print(f" Path not found: {model_path}")
        return

    zip_name = f"{model_path.name}_lora_adapter"
    zip_path = f"/tmp/{zip_name}"

    print(f" Creating zip: {model_path}")
    shutil.make_archive(zip_path, 'zip', model_path.parent, model_path.name)

    zip_file = f"{zip_path}.zip"
    print(f"   Size: {os.path.getsize(zip_file)/(1024*1024):.1f} MB")

    try:
        from google.colab import files
        print(" Downloading...")
        files.download(zip_file)
    except ImportError:
        print(f"\n Download manually: {zip_file}")

zip_and_download(DOWNLOAD_PATH)

 Creating zip: checkpoints/phase2_spider/final
   Size: 75.1 MB
 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Testing

Using encapsulated functions from `testing_utils.py`

In [51]:
from scripts.testing_utils import (
    generate_sql,
    load_finetuned_model,
    # Evaluation functions
    evaluate_model,              # Exact Match only
    evaluate_with_execution,     # Exact Match + Execution Match
    show_evaluation_examples,
    # Testing functions
    run_quick_test,
    run_interactive_test,
    test_with_university_schema,
    test_with_ecommerce_schema,
    UNIVERSITY_SCHEMA,
    ECOMMERCE_SCHEMA,
    SAMPLE_QUESTIONS,
    # EGD functions
    generate_sql_with_egd,
    evaluate_with_egd,
)
print("Testing utilities imported (including Execution Match and EGD)!")

Testing utilities imported (including Execution Match and EGD)!


### Load Fine-tuned Model

In [52]:
# Configuration
CHECKPOINT_PATH = "./checkpoints/phase2_spider/final"

# Load fine-tuned model (or use already loaded model from training)
from pathlib import Path
if Path(CHECKPOINT_PATH).exists():
    eval_model, eval_tokenizer = load_finetuned_model(CHECKPOINT_PATH)
else:
    print(f"Checkpoint not found: {CHECKPOINT_PATH}")
    print("Using model from training session...")
    eval_model, eval_tokenizer = model, tokenizer

Loading base model: Qwen/Qwen2.5-7B-Instruct
Using 4-bit quantization
Loading tokenizer...
Loading base model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading LoRA adapters...
 Model loaded successfully!


### Quick Test

In [53]:
# Run quick test with default schema/question
run_quick_test(eval_model, eval_tokenizer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


QUICK TEST

Question: How many students are majoring in Computer Science?

Schema:
[TABLES]
student:
    id (PK)
    name
    age
    major

Generated SQL:
SELECT count(*) FROM student WHERE major  =  "Computer Science"


'SELECT count(*) FROM student WHERE major  =  "Computer Science"'

### Evaluate on Spider Dev Set

In [55]:
# Evaluate on Spider dev set
EVAL_MAX_SAMPLES = 100  # Set to None for full evaluation
USE_EGD = False          # Use Execution-Guided Decoding for better accuracy
EGD_CANDIDATES = 5      # Number of candidates for EGD

if spider_dev:
    eval_results = evaluate_model(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EVAL_MAX_SAMPLES,
        use_egd=USE_EGD,
        egd_candidates=EGD_CANDIDATES,
    )

    # Show example predictions
    show_evaluation_examples(eval_results, num_correct=3, num_incorrect=3)
else:
    print("Spider dev data not loaded.")

Evaluating on 100 samples (Standard mode)...
  [10/100] Accuracy: 70.0%
  [20/100] Accuracy: 70.0%
  [30/100] Accuracy: 56.7%
  [40/100] Accuracy: 45.0%
  [50/100] Accuracy: 42.0%
  [60/100] Accuracy: 38.3%
  [70/100] Accuracy: 35.7%
  [80/100] Accuracy: 35.0%
  [90/100] Accuracy: 31.1%
  [100/100] Accuracy: 28.0%

 Final Accuracy: 28.00% (28/100)
CORRECT PREDICTIONS

[1] Question: How many singers do we have?
    SQL: SELECT count(*) FROM singer

[2] Question: What is the total number of singers?
    SQL: SELECT count(*) FROM singer

[3] Question: What are the names, countries, and ages for every singer in descending order of age?
    SQL: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

INCORRECT PREDICTIONS

[1] Question: Show name, country, age for all singers ordered by age from the oldest to the youngest.
    Gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC
    Pred: SELECT Name ,  Country ,  Age FROM singer ORDER BY Age

[2] Question: Show the name a

### Evaluate with Execution Match (EM + EX)

Execution Match compares the actual execution results of predicted SQL vs gold SQL on a mock database. This is more forgiving than Exact Match since different SQL queries can produce the same results.


In [57]:
# Evaluate with both Exact Match (EM) and Execution Match (EX)
# This requires DuckDB: pip install duckdb
# Uses EGD setting from above cell

if spider_dev:
    ex_results = evaluate_with_execution(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EVAL_MAX_SAMPLES,
        verbose=True,
        use_egd=USE_EGD,
        egd_candidates=EGD_CANDIDATES,
    )

    print("\n" + "=" * 60)
    print("SUMMARY" + (" (with EGD)" if USE_EGD else ""))
    print("=" * 60)
    print(f"Exact Match (EM):     {ex_results['exact_match_accuracy']:.2f}%")
    print(f"Execution Match (EX): {ex_results['execution_match_accuracy']:.2f}%")

    # Show how many queries EX found correct that EM missed
    ex_only = sum(1 for r in ex_results['results']
                  if r['execution_match'] and not r['exact_match'])
    if ex_only > 0:
        print(f"\nEX found {ex_only} additional correct queries beyond EM!")
else:
    print("Spider dev data not loaded.")


Evaluating 100 samples with EM + EX (Standard mode)...
------------------------------------------------------------
  [10/100] EM: 70.0%, EX: 60.0%
  [20/100] EM: 70.0%, EX: 50.0%
  [30/100] EM: 56.7%, EX: 53.3%
  [40/100] EM: 45.0%, EX: 52.5%
  [50/100] EM: 42.0%, EX: 52.0%
  [60/100] EM: 38.3%, EX: 56.7%
  [70/100] EM: 35.7%, EX: 54.3%
  [80/100] EM: 35.0%, EX: 52.5%
  [90/100] EM: 31.1%, EX: 52.2%
  [100/100] EM: 28.0%, EX: 52.0%
------------------------------------------------------------
Final Results:
  Exact Match (EM):     28.00% (28/100)
  Execution Match (EX): 52.00% (52/100)
  Execution Errors:     35 (schema parsing issues)

  EX found 37 additional correct queries beyond EM

SUMMARY
Exact Match (EM):     28.00%
Execution Match (EX): 52.00%

EX found 37 additional correct queries beyond EM!


### Interactive Testing

In [ ]:
# Test with university schema
test_with_university_schema(eval_model, eval_tokenizer)

In [ ]:
# Test with ecommerce schema
test_with_ecommerce_schema(eval_model, eval_tokenizer)

In [ ]:
# Custom test - modify schema and questions as needed
custom_schema = """[TABLES]
employee:
    id (PK)
    name
    department
    salary"""

custom_questions = [
    "What is the average salary?",
    "List employees in the Engineering department.",
    "Who has the highest salary?",
]

run_interactive_test(eval_model, eval_tokenizer, custom_schema, custom_questions)

interactive testing with EGD

In [ ]:
# Test EGD on a single example
test_schema = UNIVERSITY_SCHEMA
test_question = "How many students are enrolled in each course?"

egd_result = generate_sql_with_egd(
    eval_model, eval_tokenizer,
    test_question,
    test_schema,
    num_candidates=5,
    verbose=True
)

print(f"\nFinal SQL: {egd_result['sql']}")
print(f"Selection method: {egd_result['method']}")

In [ ]:
# Evaluate with EGD on Spider dev set
# EGD generates multiple candidates and selects the best executable one
EGD_EVAL_SAMPLES = 50  # Fewer samples since EGD is slower

if spider_dev:
    egd_results = evaluate_with_egd(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EGD_EVAL_SAMPLES,
        num_candidates=5,
        verbose=True
    )

    print("\n" + "=" * 60)
    print("EGD EVALUATION SUMMARY")
    print("=" * 60)
    print(f"Exact Match (EM):     {egd_results['exact_match_accuracy']:.2f}%")
    print(f"Execution Match (EX): {egd_results['execution_match_accuracy']:.2f}%")
    print(f"EGD Execution Rate:   {egd_results['execution_rate']:.2f}%")
else:
    print("Spider dev data not loaded.")

## Possible Improvement directions



1. **Input Format**: WikiSQL model expects `translate English to SQL: <question> context: <columns>`

2. **More Data**: The sample dataset is tiny. Use hundreds or thousands of examples for real training.

3. **LoRA Rank**: Start with r=16. Increase to 32 or 64 if underfitting, decrease to 8 if overfitting.

4. **Learning Rate**: 1e-4 to 3e-4 typically works well for LoRA.